In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, lit, avg, when, broadcast

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
jobs_df = spark.table("database.jobs")
popu_df = spark.table("database.population")

jobs_df = jobs_df.filter(col("indicador_name").like("%Unemployment with inter%"))
jobs_df = jobs_df.drop(col("pais_cod"), col("indicador_code"))

popu_df = popu_df.filter(col("cod_indicador").like("%SP.POP.TOTL%"))
popu_df = popu_df.drop(col("cod_pais"), col("cod_indicador"), col("nome_indicador"), col("Country_name"), col("Country_code"), col("Indicator_name"), col("Indicator_code"))
popu_df = popu_df.withColumnRenamed("valor", "valorI")
popu_df = popu_df.withColumnRenamed("nome_pais", "pais_name")
popu_df = popu_df.withColumn("ano", col("ano").cast(IntegerType()))

In [3]:
paises = ["%Liechtenstein%", "%Germany%", "%Denmark%", "%Switzerland%", "%Poland%", "%Czech%", 
          "%Slovakia%", "%Slovenia%", "%Hungary%", "%Austria%"]
anos = ["2010", "2011", "2012", "2013", "2014", "2015"]

jobs_df = jobs_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3]) |
    col("pais_name").like(paises[4]) |
    col("pais_name").like(paises[5]) |
    col("pais_name").like(paises[6]) |
    col("pais_name").like(paises[7]) |
    col("pais_name").like(paises[8]) |
    col("pais_name").like(paises[9])
).filter(
    col("ano").isin(anos) &
    col("valor").isNotNull()
)

popu_df = popu_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3]) |
    col("pais_name").like(paises[4]) |
    col("pais_name").like(paises[5]) |
    col("pais_name").like(paises[6]) |
    col("pais_name").like(paises[7]) |
    col("pais_name").like(paises[8]) |
    col("pais_name").like(paises[9])
).filter(
    col("ano").isin(anos) &
    col("valorI").isNotNull()
)

In [4]:
literacia_df = jobs_df.join(popu_df, ["pais_name", "ano"], "left")
literacia_df = literacia_df.withColumn("valorI", ((col("valor") / 100) * (col("valorI") / 2)).cast("int"))
literacia_df = literacia_df.withColumnRenamed("pais_name", "paises")

In [5]:
literacia_df = literacia_df.withColumn("genero",
    when(col("indicador_name").like("% female%"), "F")
    .when(col("indicador_name").like("% male%"), "M")
)

literacia_df = literacia_df.filter(col("genero").isNotNull())

literacia_df = literacia_df.drop(col("indicador_name"))

In [6]:
literacia_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaEnrolment/")

In [7]:
spark.sql(
    """
    SELECT * FROM database.EnrolmentGold
    """
).toPandas()

,paises,ano,valor,valorI,genero
0,Denmark,2010,7.5119,208368,F
1,Denmark,2011,6.8015,189441,F
2,Denmark,2012,6.6001,184524,F
3,Denmark,2013,5.4890,154101,F
4,Denmark,2014,5.2409,147884,F
5,Denmark,2015,4.9660,141120,F
6,Denmark,2010,6.8138,189004,M
7,Denmark,2011,6.7084,186848,M
8,Denmark,2012,6.8052,190258,M
9,Denmark,2013,6.2315,174947,M


In [8]:
spark.stop()